In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('explain_plan').getOrCreate()
# spark.conf.set("spark.sql.adaptive.enabled", "true")

23/12/13 22:23:38 WARN Utils: Your hostname, SAURAVs-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.29.194 instead (on interface en0)
23/12/13 22:23:38 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/13 22:23:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
n = spark.range(1, 1000000)
times5 = n.selectExpr("id * 5 as id")

In [3]:
n.show(3)

+---+
| id|
+---+
|  1|
|  2|
|  3|
+---+
only showing top 3 rows



In [5]:
times5.show(3)

+---+
| id|
+---+
|  5|
| 10|
| 15|
+---+
only showing top 3 rows



In [8]:
times5.explain(mode = 'simple')

== Physical Plan ==
*(1) Project [(id#0L * 5) AS id#2L]
+- *(1) Range (1, 1000000, step=1, splits=8)




In [9]:
times5.explain(mode = 'extended') #extended=True

== Parsed Logical Plan ==
'Project [('id * 5) AS id#2]
+- Range (1, 1000000, step=1, splits=Some(8))

== Analyzed Logical Plan ==
id: bigint
Project [(id#0L * cast(5 as bigint)) AS id#2L]
+- Range (1, 1000000, step=1, splits=Some(8))

== Optimized Logical Plan ==
Project [(id#0L * 5) AS id#2L]
+- Range (1, 1000000, step=1, splits=Some(8))

== Physical Plan ==
*(1) Project [(id#0L * 5) AS id#2L]
+- *(1) Range (1, 1000000, step=1, splits=8)



In [12]:
m = spark.range(1, 1000000, 2)
split7 = m.repartition(7)
split7.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Exchange RoundRobinPartitioning(7), REPARTITION_BY_NUM, [plan_id=50]
   +- Range (1, 1000000, step=2, splits=8)




In [14]:
split7.show(3)

+------+
|    id|
+------+
|106581|
| 99177|
| 64689|
+------+
only showing top 3 rows



In [13]:
m.explain()

== Physical Plan ==
*(1) Range (1, 1000000, step=2, splits=8)




In [20]:
ds1 = spark.range(1, 100)
ds2 = spark.range(1, 100, 2)
ds3 = ds1.repartition(7)
ds4 = ds2.repartition(9)
ds5 = ds3.selectExpr("id * 5 as id")
joined = ds5.join(ds4, "id")
sum = joined.selectExpr("sum(id)")
sum.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[], functions=[sum(id#83L)])
   +- Exchange SinglePartition, ENSURE_REQUIREMENTS, [plan_id=226]
      +- HashAggregate(keys=[], functions=[partial_sum(id#83L)])
         +- Project [id#83L]
            +- BroadcastHashJoin [id#83L], [id#77L], Inner, BuildRight, false
               :- Project [(id#75L * 5) AS id#83L]
               :  +- Exchange RoundRobinPartitioning(7), REPARTITION_BY_NUM, [plan_id=212]
               :     +- Range (1, 100, step=1, splits=8)
               +- BroadcastExchange HashedRelationBroadcastMode(List(input[0, bigint, false]),false), [plan_id=221]
                  +- Exchange RoundRobinPartitioning(9), REPARTITION_BY_NUM, [plan_id=215]
                     +- Range (1, 100, step=2, splits=8)




In [17]:
sum.explain(True)

== Parsed Logical Plan ==
'Project [unresolvedalias('sum('id), Some(org.apache.spark.sql.Column$$Lambda$3080/0x00000008003cae78@ec47cd8))]
+- Project [id#41L]
   +- Join Inner, (id#41L = id#35L)
      :- Project [(id#33L * cast(5 as bigint)) AS id#41L]
      :  +- Repartition 7, true
      :     +- Range (1, 1000000, step=1, splits=Some(8))
      +- Repartition 9, true
         +- Range (1, 1000000, step=2, splits=Some(8))

== Analyzed Logical Plan ==
sum(id): bigint
Aggregate [sum(id#41L) AS sum(id)#45L]
+- Project [id#41L]
   +- Join Inner, (id#41L = id#35L)
      :- Project [(id#33L * cast(5 as bigint)) AS id#41L]
      :  +- Repartition 7, true
      :     +- Range (1, 1000000, step=1, splits=Some(8))
      +- Repartition 9, true
         +- Range (1, 1000000, step=2, splits=Some(8))

== Optimized Logical Plan ==
Aggregate [sum(id#41L) AS sum(id)#45L]
+- Project [id#41L]
   +- Join Inner, (id#41L = id#35L)
      :- Project [(id#33L * 5) AS id#41L]
      :  +- Repartition 7, true
  

In [21]:
sum.explain(mode="formatted")

== Physical Plan ==
AdaptiveSparkPlan (12)
+- HashAggregate (11)
   +- Exchange (10)
      +- HashAggregate (9)
         +- Project (8)
            +- BroadcastHashJoin Inner BuildRight (7)
               :- Project (3)
               :  +- Exchange (2)
               :     +- Range (1)
               +- BroadcastExchange (6)
                  +- Exchange (5)
                     +- Range (4)


(1) Range
Output [1]: [id#75L]
Arguments: Range (1, 100, step=1, splits=Some(8))

(2) Exchange
Input [1]: [id#75L]
Arguments: RoundRobinPartitioning(7), REPARTITION_BY_NUM, [plan_id=212]

(3) Project
Output [1]: [(id#75L * 5) AS id#83L]
Input [1]: [id#75L]

(4) Range
Output [1]: [id#77L]
Arguments: Range (1, 100, step=2, splits=Some(8))

(5) Exchange
Input [1]: [id#77L]
Arguments: RoundRobinPartitioning(9), REPARTITION_BY_NUM, [plan_id=215]

(6) BroadcastExchange
Input [1]: [id#77L]
Arguments: HashedRelationBroadcastMode(List(input[0, bigint, false]),false), [plan_id=221]

(7) BroadcastHashJoin


In [31]:
data1 = [
    ['1', 'Java', '20000'],
    ['2', 'Python', '30000'],
    ['3', 'Scala', '3000']
]
schema1 = ['id', 'language', 'fees']
languages = spark.createDataFrame(data1, schema1)
languages.createOrReplaceTempView("languages")

data2 = [
    ['1', 'StudentA'],
    ['2', 'StudentB'],
    ['3', 'StudentA'],
    ['1', 'StudentC'],
    ['3', 'StudentB']
]
schema2 = ['language_id', 'studentName']
students = spark.createDataFrame(data2, schema2)
students.createOrReplaceTempView("students")

In [42]:
df = spark.sql("""SELECT students.studentName, SUM(students.language_id) as sum
               FROM students
               INNER JOIN languages
               ON students.language_id = languages.id
               WHERE students.studentName = 'StudentC' 
               group By students.studentName
               """)

In [43]:
df.show()

+-----------+---+
|studentName|sum|
+-----------+---+
|   StudentC|1.0|
+-----------+---+



In [44]:
df.explain(True)

== Parsed Logical Plan ==
'Aggregate ['students.studentName], ['students.studentName, 'SUM('students.language_id) AS sum#294]
+- 'Filter ('students.studentName = StudentC)
   +- 'Join Inner, ('students.language_id = 'languages.id)
      :- 'UnresolvedRelation [students], [], false
      +- 'UnresolvedRelation [languages], [], false

== Analyzed Logical Plan ==
studentName: string, sum: double
Aggregate [studentName#186], [studentName#186, sum(cast(language_id#185 as double)) AS sum#294]
+- Filter (studentName#186 = StudentC)
   +- Join Inner, (language_id#185 = id#179)
      :- SubqueryAlias students
      :  +- View (`students`, [language_id#185,studentName#186])
      :     +- LogicalRDD [language_id#185, studentName#186], false
      +- SubqueryAlias languages
         +- View (`languages`, [id#179,language#180,fees#181])
            +- LogicalRDD [id#179, language#180, fees#181], false

== Optimized Logical Plan ==
Aggregate [studentName#186], [studentName#186, sum(cast(language_id